In [ ]:
import os
import sys
from pathlib import Path

sys.path.insert(0, '..')

import numpy as np
import pandas as pd
import scipy as sp
from scipy import sparse

from lightfm import LightFM

## DUMMY DATA

In [96]:
mat = sparse.lil_matrix((10, 5), dtype=np.int32)
mat[1, 2] = 1
mat[2, 2] = 1
mat[3, 1] = 1
mat[5, 2] = 1
mat[6, 0] = 1
mat[8, 3] = 1
mat[9, 4] = 1
mat = mat.tocoo()

cat = sparse.lil_matrix((12, 6), dtype=np.int32)
cat[0, 0] = 3
cat[1, 5] = 1
cat[2, 3] = 1
cat[3, 1] = 1
cat[4, 2] = 1
cat[5, 3] = 1
cat[6, 4] = 1
cat[7, 1] = 1
cat[8, 2] = 1
cat[9, 3] = 1
cat[10, 4] = 1
cat[11, 5] = 1
cat[11, 3] = 1
cat = cat.tocoo()

In [97]:
# Users = rows
# Items = cols
pd.DataFrame.sparse.from_spmatrix(mat)

,0,1,2,3,4
0,0,0,0,0,0
1,0,0,1,0,0
2,0,0,1,0,0
3,0,1,0,0,0
4,0,0,0,0,0
5,0,0,1,0,0
6,1,0,0,0,0
7,0,0,0,0,0
8,0,0,0,1,0
9,0,0,0,0,1


In [98]:
pd.DataFrame.sparse.from_spmatrix(cat)

,0,1,2,3,4,5
0,3,0,0,0,0,0
1,0,0,0,0,0,1
2,0,0,0,1,0,0
3,0,1,0,0,0,0
4,0,0,1,0,0,0
5,0,0,0,1,0,0
6,0,0,0,0,1,0
7,0,1,0,0,0,0
8,0,0,1,0,0,0
9,0,0,0,1,0,0


## TRAIN

In [99]:
model = LightFM(no_components=2, loss='warp', n=10)
LightFM.fit_partial(model, interactions=mat, epochs=10, num_threads=2, first_unlocked_item_id=0, first_unlocked_user_id=0)

In [100]:
_, reps = model.get_user_representations()
u1 = pd.DataFrame(reps).copy()

_, reps = model.get_item_representations()
i1 = pd.DataFrame(reps).copy()

In [101]:
# Resize model to accomodate new size
model.resize(cat)

In [102]:
# First_unlocked_id 0,1,2 = 3
LightFM.fit_partial(model, interactions=cat, epochs=30, num_threads=2, first_unlocked_item_id=3, first_unlocked_user_id=3)

In [103]:
_, reps = model.get_user_representations()
u2 = pd.DataFrame(reps).copy()

_, reps = model.get_item_representations()
i2 = pd.DataFrame(reps).copy()

## COMPARE

In [104]:
#changes in these USERS
pd.DataFrame(pd.concat([u1,u2]).drop_duplicates(keep=False).reset_index()['index'].drop_duplicates())

,index
0,3
1,4
2,5
3,6
4,7
5,8
6,9
14,10
15,11


In [105]:
#changes in these ITEMS
pd.DataFrame(pd.concat([i1,i2]).drop_duplicates(keep=False).reset_index()['index'].drop_duplicates())

,index
0,3
1,4
4,5


In [106]:
i1

,0,1
0,-0.170198,0.083651
1,0.185602,0.224149
2,-0.358000,0.339907
3,0.146484,0.025660
4,-0.057875,-0.265332


In [107]:
i2

,0,1
0,-0.170198,0.083651
1,0.185602,0.224149
2,-0.358000,0.339907
3,-0.844618,0.806573
4,0.270408,-1.465973
5,-1.878266,-0.009871


In [108]:
u1

,0,1
0,0.213175,-0.187784
1,-0.241411,0.172417
2,0.058361,0.129113
3,0.270734,0.200279
4,0.232715,0.079264
5,-0.306679,0.195146
6,0.041891,-0.048172
7,-0.188227,-0.199616
8,0.157742,0.113392
9,-0.006165,-0.118249


In [109]:
u2

,0,1
0,0.213175,-0.187784
1,-0.241411,0.172417
2,0.058361,0.129113
3,1.482984,0.458277
4,0.086803,0.615755
5,-0.134406,0.994197
6,0.543554,-0.793747
7,1.052534,0.429633
8,0.016110,0.641742
9,-0.040868,0.655340
